In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

In [2]:
response = requests.get('https://freelinuxpcgames.com/all-games/')

In [3]:
soup = BeautifulSoup(response.text,'html.parser')

In [4]:
data_lis = []
for i in soup.select('.az-columns a'):
    data = {
        'title': i.text.strip(),
        'link': i['href']
    }
    data_lis.append(data)

In [5]:
df = pd.DataFrame(data_lis)

In [7]:
df.link[0]

'https://freelinuxpcgames.com/abandon-ship/'

In [8]:
response = requests.get(df.link[0])
soup = BeautifulSoup(response.text, 'html.parser')

'File Size: 5.23 GB'

In [23]:
games = []

# tqdm for loop progress
for row in tqdm(df.itertuples(index=False), total=len(df)):
    game_info = {
        'link': row.link,
        'title': row.title
    }
    try:
        response = requests.get(row.link)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        file_size = soup.select_one('.nv-content-wrap.entry-content p[style="text-align: center;"]').text
        game_info['file_size'] = file_size
        
        for p in soup.select('.nv-content-wrap p'):
            if 'Genre :' in p.text:
                lines = [line.strip() for line in p.decode_contents().split("<br/>")]
                for line in lines:
                    if ":" in line:
                        key, value = line.split(":", 1)
                        game_info[key.strip()] = value.strip()
                break

    except Exception as e:
        game_info['error'] = str(e)

    games.append(game_info)


  0%|          | 0/990 [00:00<?, ?it/s]

100%|██████████| 990/990 [08:46<00:00,  1.88it/s]


In [24]:
import re

# Step 1: Create a normalization + mapping function
def normalize_key(key):
    key = key.lower()
    key = re.sub(r'\s+', ' ', key).strip()
    key = re.sub(r'[^\w\s]', '', key)

    key_mappings = {
        'voice language': 'voice',
        'voice acting language': 'voice',
        'voiceover': 'voice',
        'voice over': 'voice',
        'voice set language': 'voice',
        'voice': 'voice',

        'interface language': 'interface',
        'interface and voice acting': 'interface_voice',
        'interface and audio language': 'interface_voice',
        'interface and voice language': 'interface_voice',
        'language interface': 'interface',
        'interface language and voice acting': 'interface_voice',

        'developer and publisher': 'developer_publisher',
        'developer & publisher': 'developer_publisher',
        'developer': 'developer',
        'publisher': 'publisher',

        'used libraries': 'libraries',
        'libraries used': 'libraries',
        'library used': 'libraries',
        'used the library': 'libraries',

        'website': 'website',
        'site': 'website',
        'game website': 'website',
        'developer site': 'website',
        'modification website': 'website',

        'version': 'version',
        'unity3d version': 'engine_version',
        'engine version': 'engine_version',
        'unity version': 'engine_version',
        'gog version': 'version',

        'developer and publisher': 'developer_publisher',
        'developer & publisher': 'developer_publisher',
        'developer amp publisher': 'developer_publisher',
        'publisher and developer': 'developer_publisher',
        'developer and publishing house': 'developer_publisher',

        'used libraries': 'libraries',
        'libraries used': 'libraries',
        'library used': 'libraries',
        'used the library': 'libraries',
        'used': 'libraries',

        'interface language': 'interface',
        'interface language ': 'interface',
        'language interface': 'interface',
        'interface and voice acting': 'interface_voice',
        'interface and voice acting language': 'interface_voice',
        'language interface and voice acting': 'interface_voice',
        'interface and audio language': 'interface_voice',

        'voice language': 'voice',
        'voice acting language': 'voice',
        'voiceover': 'voice',
        'voice over': 'voice',
        'speech language': 'voice',
        'audio language': 'voice',
        'voice acting': 'voice',
        'language voice acting': 'voice',

        'website': 'website',
        'developer site': 'website',
        'site': 'website',
        'game site': 'website',
        'official site': 'website',
        'site of the game': 'website',
        'page at gogcom': 'gog_page',
        'page on gogcom': 'gog_page',
        'gogcom page': 'gog_page',

        'unity3d version': 'engine_version',
        'engine version': 'engine_version',
        'unity version': 'engine_version',
        'game engine': 'engine',
        'framework': 'engine',
        'framework used': 'engine',

        'developer website': 'developer_website',
        'engine site': 'engine_site',
        'engine website': 'engine_site',

        'tabletka': 'tablet',
        'ui amp voiceover': 'interface_voice',
        'ui': 'interface',
        'interface and voice language': 'interface_voice',
        'voice set language': 'voice',
        'language': 'interface',  # ambiguous, consolidate if it makes sense
        'platform': 'architecture',  # if that's appropriate

        'year of release': 'release_year',
        'release': 'release_year',
        'game version': 'version',
        'new version available': 'availability_of_new_version',
        'availability of a new version': 'availability_of_new_version',
        'modification website': 'website',

        'publishing house': 'publisher',
        'developer site': 'developer_website',
        'developer website': 'developer_website',
        'publisher': 'publisher',
        
    }

    return key_mappings.get(key, key)


cleaned_games = []

for game in games:
    cleaned_game = {}
    for key, val in game.items():
        if not isinstance(key, str) or len(key) > 50 or key.strip().startswith('<'):
            continue

        clean_key = normalize_key(key)
        cleaned_game[clean_key] = val.strip() if isinstance(val, str) else val

    cleaned_games.append(cleaned_game)


In [38]:
df = pd.DataFrame(cleaned_games)

In [39]:
df = df.dropna(axis=1, thresh=100)

In [27]:
for
import pandas as pd

# Assume df is your DataFrame and 'genre' is the column
def normalize_genres(genre_str):
    if pd.isna(genre_str):
        return []
    separators = ['/', '–', '-', ';', '—', '&', 'and', '’', '&amp;', '‘', '‘n’']
    for sep in separators:
        genre_str = genre_str.replace(sep, ',')
    genre_str = genre_str.lower()
    return list({g.strip() for g in genre_str.split(',') if g.strip()})

df['normalized_genres'] = df['genre'].apply(normalize_genres)

SyntaxError: invalid syntax (3063705435.py, line 1)

In [28]:
df

,link,title,file_size,genre,developer_publisher,libraries,multiplayer,architecture,version,license,interface,tablet,developer,website,publisher,engine,voice
0,https://freelinuxpcgames.com/abandon-ship/,Abandon Ship,File Size: 5.23 GB,"Strategy, Roguelike, Naval Battles",Fireblade Software,Unity3D,No,amd64,1.3.14934,Proprietary,Multilingual,Not Required,NaN,NaN,NaN,NaN,NaN
1,https://freelinuxpcgames.com/absolute-drift/,Absolute Drift (5f6049d),File Size: 149.3 MB,Racing – Simulation – Open World,NaN,NaN,no,amd64,4e17697 updated to 5f6049d,proprietary,English,not required,"Funselektor Labs Inc., FlippFly Game",https://www.gog.com/game/absolute_drift,Funselektor Labs Inc.,Unity,no
2,https://freelinuxpcgames.com/acting-lessons/,Acting Lessons,File Size: 3.39 GB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://freelinuxpcgames.com/adom/,ADOM,File Size: 525.4 MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://freelinuxpcgames.com/aer-memories-of-old/,AER Memories of Old,File Size: 658.9 MB,Action / Adventure / Mystery,NaN,Unity3D (5.6.1p2),no,"x86, amd64",1.0.3,proprietary,"Russian, English , German, Spanish, French, Ja...",NaN,Forgotten Key,NaN,Daedalic Entertainment,NaN,"English, German"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,https://freelinuxpcgames.com/7-days-to-die/,7 Days to Die (A21),File Size: 10.72 GB,"Action (Shooter), Survival, 3D, 1st Person",NaN,Unity3D,"LAN, internet",amd64,A20.5 (b2) updated to A21 (b324),proprietary,multilingual,NaN,The Fun Pimps,NaN,The Fun Pimps Entertainment LLC,NaN,no
986,https://freelinuxpcgames.com/8-bit-commando/,8-Bit Commando,File Size:,"Shooter, Platformer, Side View",2dengine,NaN,No,amd64,1.7.0,Proprietary,Multilingual,NaN,NaN,NaN,NaN,LÖVE,NaN
987,https://freelinuxpcgames.com/80-days/,80 Days,File Size: 82.8 MB,"adventure, strategy, text",NaN,NaN,no,amd64,1.17.7g,proprietary,English,NaN,"inkle Ltd , Cape Guy Ltd",NaN,inkle Ltd,"Unity * (2018.4.24f1, * unofficial build for L...",NaN
988,https://freelinuxpcgames.com/88-heroes/,88 Heroes,File Size: 19 MB,Action – Arcade – Platformer,NaN,Unity3D (5.4.3p3),NaN,"x86, amd64",N / A (2017-03-24),proprietary,"English, French, Italian, German, Spanish, Jap...",NaN,Bitmap Bureau,http://www.88heroes.com/,Rising Star Games,NaN,English


In [40]:
df['file_size']=df['file_size'].str.replace(r'File Size: ','')

In [41]:
df['file_size'] = df['file_size'].str.replace(" ", "")
def convert_to_mb(value):
        # print(value)
    # try:
        if 'GB' in value:
            return float(value.replace('GB', '')) * 1024
        elif 'MB' in value:
            return float(value.replace('MB', ''))
        # else:
        #     return None
    # except:
    #     return None

# Step 3: Apply conversion
df['file_size_MB'] = df['file_size'].apply(convert_to_mb)

In [45]:
df['file_size_MB'] = df['file_size_MB'].round(1)

In [ ]:
for i, tags_list in df['normalized_genres'].items():
    for tag in tags_list:
        df.at[i, f't-{tag}'] = 1

In [71]:
df = df.dropna(axis=1, thresh=5)

In [ ]:
for col in [c for c in df.columns if 't-' in c]:
    df[col] = df[col].fillna(0)

array([1., 0.])

In [46]:
df.to_excel('Free Linux Games.xlsx',index=False)